In [1]:
import csv
import json
import geopandas as gpd

In [2]:
file = open('../assets/migration.tsv', 'r')

In [3]:
reader = csv.DictReader(file, delimiter='\t')

In [4]:
data = [r for r in reader]

In [5]:
jsonData = []
for r in data:
    if '   ' in r['Region, development group, country or area of origin'] and '   ' in r['Region, development group, country or area of destination']:
        stringValues = [r['1990'], r['1995'], r['2000'], r['2005'], r['2010'], r['2015'], r['2020']]
        values = [int(s.replace('..', '0').replace(' ', '')) for s in stringValues]
        jsonData.append({
            'origin': r['Region, development group, country or area of origin'].strip().replace('*', ''),
            'destination': r['Region, development group, country or area of destination'].strip().replace('*', ''),
            'timeseries': values
        })

In [6]:
countriesDf = gpd.read_file('../assets/custom.geo.json')

In [7]:
countryName2DataName = {
    'United States': 'United States of America',
    'Russia': 'Russian Federation',
    'Dominican Rep.': 'Dominican Republic',
    'Venezuela': 'Venezuela (Bolivarian Republic of)',
    'Bolivia': 'Bolivia (Plurinational State of)',
    'Falkland Is.': 'Falkland Islands (Malvinas)',
    'Brunei': 'Brunei Darussalam',
    'N. Cyprus': 'Cyprus',
    'Iran': 'Iran (Islamic Republic of)',
    'Korea': 'Republic of Korea',
    'Lao PDR': "Lao People's Democratic Republic",
    'Dem. Rep. Korea': "Dem. People's Republic of Korea",
    'Palestine': 'State of Palestine',
    'Syria': 'Syrian Arab Republic',
    'Taiwan': 'China, Taiwan Province of China',
    'Vietnam': 'Viet Nam',
    'Central African Rep.': 'Central African Republic',
    'Dem. Rep. Congo': 'Democratic Republic of the Congo',
    'Eq. Guinea': 'Equatorial Guinea',
    'W. Sahara': 'Western Sahara',
    'S. Sudan': 'South Sudan',
    'Somaliland': 'Somalia',
    'Swaziland': 'Eswatini',
    'Tanzania': 'United Republic of Tanzania',
    'Bosnia and Herz.': 'Bosnia and Herzegovina',
    'Czech Rep.': 'Czechia',
    'Kosovo': '',
    'Macedonia': 'North Macedonia',
    'Moldova': 'Republic of Moldovia',
    'Solomon Is.': 'Solomon Islands',
}

In [8]:
countriesDf = countriesDf[['name', 'name_long', 'postal', 'formal_en', 'pop_est', 'gdp_md_est', 'pop_year', 'lastcensus', 'gdp_year', 'economy', 'income_grp', 'geometry']]
countriesDf['alt_name'] = countriesDf['name'].map(lambda n: countryName2DataName[n] if n in countryName2DataName else n)
countriesDf['emigration'] = countriesDf['alt_name'].map(lambda n: [entry for entry in jsonData if entry['origin'] == n])
countriesDf['centroid'] = countriesDf.centroid.map(lambda c: [c.x, c.y])
countriesDf.insert(0, 'id', range(len(countriesDf)))
countriesDf


/tmp/ipykernel_15169/1629137834.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  countriesDf['centroid'] = countriesDf.centroid.map(lambda c: [c.x, c.y])


,name,name_long,postal,formal_en,pop_est,gdp_md_est,pop_year,lastcensus,gdp_year,economy,income_grp,geometry,alt_name,emigration,centroid
0,Bahamas,Bahamas,BS,Commonwealth of the Bahamas,309156,9093.0,-99,2010,-99,6. Developing region,2. High income: nonOECD,"MULTIPOLYGON (((-77.53466 23.75975, -77.78000 ...",Bahamas,"[{'origin': 'Bahamas', 'destination': 'Egypt',...","[-77.92997080393516, 25.515491725336627]"
1,Belize,Belize,BZ,Belize,307899,2536.0,-99,2010,-99,6. Developing region,4. Lower middle income,"POLYGON ((-89.14308 17.80832, -89.15091 17.955...",Belize,"[{'origin': 'Belize', 'destination': 'Guinea',...","[-88.7034212529932, 17.197089911451542]"
2,Canada,Canada,CA,Canada,33487208,1300000.0,-99,2011,-99,1. Developed region: G7,1. High income: OECD,"MULTIPOLYGON (((-63.66450 46.55001, -62.93930 ...",Canada,"[{'origin': 'Canada', 'destination': 'Mauritiu...","[-98.14238137209692, 61.469076145348886]"
3,Costa Rica,Costa Rica,CR,Republic of Costa Rica,4253877,48320.0,-99,2011,-99,5. Emerging region: G20,3. Upper middle income,"POLYGON ((-82.96578 8.22503, -83.50844 8.44693...",Costa Rica,"[{'origin': 'Costa Rica', 'destination': 'Guin...","[-84.17542309600944, 9.965671127464525]"
4,Cuba,Cuba,CU,Republic of Cuba,11451652,108200.0,-99,2002,-99,5. Emerging region: G20,3. Upper middle income,"POLYGON ((-82.26815 23.18861, -81.40446 23.117...",Cuba,"[{'origin': 'Cuba', 'destination': 'Seychelles...","[-78.96068490970252, 21.63175154102521]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,New Caledonia,New Caledonia,NC,New Caledonia,227436,3158.0,-99,2009,-99,6. Developing region,2. High income: nonOECD,"POLYGON ((165.77999 -21.08000, 166.59999 -21.7...",New Caledonia,"[{'origin': 'New Caledonia', 'destination': 'H...","[165.5344746008754, -21.26135761252651]"
171,New Zealand,New Zealand,NZ,New Zealand,4213418,116700.0,-99,2006,-99,2. Developed region: nonG7,1. High income: OECD,"MULTIPOLYGON (((173.02037 -40.91905, 173.24723...",New Zealand,"[{'origin': 'New Zealand', 'destination': 'Egy...","[172.70192594405583, -41.662578757158684]"
172,Papua New Guinea,Papua New Guinea,PG,Independent State of Papua New Guinea,6057263,13210.0,-99,2000,-99,6. Developing region,4. Lower middle income,"MULTIPOLYGON (((155.88003 -6.82000, 155.59999 ...",Papua New Guinea,"[{'origin': 'Papua New Guinea', 'destination':...","[145.31757462782244, -6.451644514630341]"
173,Solomon Is.,Solomon Islands,SB,None,595613,1078.0,-99,2009,-99,7. Least developed region,4. Lower middle income,"MULTIPOLYGON (((162.11902 -10.48272, 162.39865...",Solomon Islands,"[{'origin': 'Solomon Islands', 'destination': ...","[159.96661544742977, -8.85249747084859]"


In [9]:
jsonData = countriesDf.to_json(drop_id=True)
fileHandle = open('../assets/countries.geo.json', 'w', encoding='utf8')
fileHandle.write(jsonData)